In [ ]:
# EMA_Crossover_Test.py
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Ticker symbol and data download
data_to_use = 'SPY'
data = yf.download(data_to_use, start='2022-01-01')
data.columns = data.columns.droplevel(1)  # drop MultiIndex if exists

def calculate_returns(quantity, data): #reused from other demo
    total_profit = 0
    total_money_invested = 0

    buy_signal = (data['Signal'] == 1.0)
    sell_signal = (data['Signal'] == -1.0)

    for buy_time, buy_row in data[buy_signal].iterrows():
        buy_price = buy_row['Close']
        sell_trades = data[sell_signal & (data.index > buy_time)]

        if not sell_trades.empty:
            sell_price = sell_trades.iloc[0]['Close']
            profit = quantity * (sell_price - buy_price)
            money_invested = quantity * buy_price
            print(f"Bought {quantity} share(s) at ${buy_price:.2f}. "
                  f"Sold {quantity} share(s) at ${sell_price:.2f}, Profit: ${profit:.2f}")

            total_profit += profit
            total_money_invested += money_invested
        else:
            print(f"Would buy {quantity} share(s) at ${buy_price:.2f}, but no sell signal found.")

    roi = (total_profit / total_money_invested) if total_money_invested != 0 else 0
    print(f"\nTotal Profit: ${total_profit:.2f}, "
          f"Money Invested: ${total_money_invested:.2f}, "
          f"Return on Investment: {roi*100:.2f}%")
    return total_profit, roi


def plot_data(data, data_name=data_to_use): #reused from other demo
    plt.figure(figsize=(12,6))
    plt.plot(data['Close'], label=f"{data_name} Close Price", alpha=0.5)
    plt.title(f"{data_name} Closing Prices Since 2022")
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')

# Calculate 12/26 EMAs
data['EMA12'] = data['Close'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['Close'].ewm(span=26, adjust=False).mean()

# Generate signals
data['Position'] = np.where(data['EMA12'] > data['EMA26'], 1, 0)
data['Signal'] = data['Position'].diff()

C:\Users\Evan Schreiner\AppData\Local\Temp\ipykernel_40516\3121469930.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(data_to_use, start='2022-01-01')
[*********************100%***********************]  1 of 1 completed


In [2]:
data

Price,Close,High,Low,Open,Volume,EMA12,EMA26,Position,Signal
Date,,,,,,,,,
2022-01-03,453.210388,453.343222,449.548364,451.872697,72668200,453.210388,453.210388,0,NaN
2022-01-04,453.058563,455.363961,451.189595,454.642929,71178700,453.187030,453.199142,0,0.0
2022-01-05,444.358856,453.466520,444.263979,452.688567,104538900,451.828850,452.544306,0,0.0
2022-01-06,443.941437,446.673739,441.560154,443.894013,86858900,450.615402,451.907056,0,0.0
2022-01-07,442.186310,445.136827,440.820159,443.950934,85111600,449.318618,451.187001,0,0.0
...,...,...,...,...,...,...,...,...,...
2025-10-08,673.109985,673.210022,669.419983,670.250000,60702200,666.836116,660.228413,1,0.0
2025-10-09,671.159973,673.940002,669.210022,673.530029,66501900,667.501325,661.038158,1,0.0
2025-10-10,653.020020,673.950012,652.840027,672.130005,159422600,665.273432,660.444222,1,0.0


In [3]:
# Create entry signals where Signal is 1 (buy)
entries = data['Signal'] == 1

# Create exit signals where Signal is -1 (sell)
exits = data['Signal'] == -1

In [5]:
import vectorbt as vbt
# Run the Vectorbt backtest
portfolio = vbt.Portfolio.from_signals(
    close=data['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,  # Start with $100,000
    freq='D'           # Use daily frequency for calculations
)

# Print the performance statistics
print("\n--- Backtest Performance ---")
print(portfolio.stats())


--- Backtest Performance ---
Start                                2022-01-03 00:00:00
End                                  2025-10-14 00:00:00
Period                                 949 days 00:00:00
Start Value                                     100000.0
End Value                                  145199.468452
Total Return [%]                               45.199468
Benchmark Return [%]                           46.119771
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               12.291187
Max Drawdown Duration                  226 days 00:00:00
Total Trades                                          12
Total Closed Trades                                   11
Total Open Trades                                      1
Open Trade PnL                              21641.433125
Win Rate [%]                                   36.363636
Best Trade [%]                                 15.984513
W